# Exploratory Data Analysis of Rotten Tomatoes Top 100 in each Genre
Rotten Tomato dataset was retrived from kaggle @ 
https://www.kaggle.com/datasets/prasertk/top-100-rotten-tomatoes-movies-by-genres

In this analysis I have tried to answer the following questions: 
1. Explore the year that had the most popular titles
2. What correlations, if any, exist?  
3. Find the most popular movies in each genre
4. The most popular movie overall

In [ ]:
!pip install plotly

In [ ]:
# Importing libraries 
import numpy as np 
import pandas as pd
import matplotlib. pyplot as plt 
%matplotlib inline

import seaborn as sns 
import plotly.express as px

In [ ]:
rt_df = pd.read_csv('top_100_movies_by_genres.csv')

# Data wrangling 
Now I will spend some time understading the values within my dataset, cleaning as necessary, and preparing it for further evaluation. 

In [ ]:
rt_df.head()

In [ ]:
rt_df.shape

In [ ]:
rt_df.info()

In [ ]:
rt_df.isnull().sum()

In [ ]:
rt_df.columns

My dataframe has 1612 rows and five columns. It has no missing values. From here, I will begin to analyze this data with descriptive statistics.


I will focus on the following columns:
1. RankingTomatometer: This dataset only includes the rating that was given to it by critics
2. No. of Reviews: The number of critics who gave their opinions on this dataset. 

In [ ]:
rt_df['No. of Reviews'].min()

In [ ]:
rt_df['No. of Reviews'].max()

In [ ]:
rt_df['No. of Reviews'].mean()

Most movies have around 148 reviews. Now I will check my data for outliers.

In [ ]:
fig1 = px.histogram(rt_df, x='No. of Reviews')
fig1.show()

This distribution is not normal, so I will be adding a new column with this information normalized through min/max scaling. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

rt_df[['Scaled Reviews']] = scaler.fit_transform(rt_df[["No. of Reviews"]])

rt_df.head()

In [ ]:
fig2 = px.histogram(rt_df, x="Scaled Reviews")
fig2.show()

The other column I would like to cover with some statistical analysis is classified as an 'object' type rather than an integer. I need to correct this in order to preform calculations.

In [ ]:
rt_df['RatingTomatometer'] = rt_df["RatingTomatometer"].str.rstrip("%").astype('float')/100.0

In [ ]:
rt_df.head()

In [ ]:
rt_df.info()

In [ ]:
rt_df["RatingTomatometer"].min()

In [ ]:
rt_df["RatingTomatometer"].max()

In [ ]:
rt_df["RatingTomatometer"].mean()

This dataset specifically focuses on the top 100 in each genre so I expect the mean to be high. 

(I want to seperate this out by genre, by movie data, and limit it down to the top 10 movies in each genre) 

In [ ]:
fig3 = px.histogram(rt_df, x="RatingTomatometer")
fig3.show()

# Question 1: What year had the most popular titles?
I will start by seperating out titles and release dates

In [ ]:
new = rt_df["Title"].str.split("(", n = 1, expand=True)

rt_df["Movie Title"] = new[0]
rt_df["Year"] = new[1]

rt_df.head()

In [ ]:
rt_df["Year"].isnull().sum()

In [ ]:
rt_df.info()

In [ ]:
rt_df["Year"].unique()

I realize that I have an error here as some titles also listed some additional information in paraenthesis after the title, which is where I seperated out the values. Since I only want numbers in this column, I will now remove some those characters. 

In [ ]:
def numbers(element):
    
    return "".join(filter(str.isnumeric, element))

In [ ]:
rt_df.loc[:, "YearB"] = [numbers(x) for x in rt_df.Year]
rt_df.head()

In [ ]:
rt_df["YearB"].unique()

In [ ]:
rt_df["YearB"].isnull().sum()

In [ ]:
rt_df.drop(["Year", "Title"], axis=1, inplace=True)

In [ ]:
rt_df.head()

I still have some values that don't make sense in my unique list above so I'm going to double check those values now.

In [ ]:
rt_df.loc[rt_df["YearB"] == "19841984"]

In [ ]:
rt_df.at[90, "YearB"]="1984"
rt_df.at[483, "YearB"]="1984"
rt_df.at[1282, "YearB"]="1984"

In [ ]:
rt_df.loc[rt_df["YearB"] == "1202017"]

In [ ]:
rt_df.at[229, "YearB"]="2017"

In [ ]:
rt_df.loc[rt_df["YearB"] == "1742003"]

In [ ]:
rt_df.at[1381, "YearB"]="2003"

In [ ]:
rt_df["YearB"].unique()

In [ ]:
rt_df.sort_values("YearB")

In [ ]:
fig4 = px.histogram(rt_df, x="YearB")
fig4.show()

In [ ]:
rt_df["YearB"].value_counts()

In [ ]:
rt_df["YearB"].max()

The year with the most popular titles was 2018

# Seperate out dataframe by Specific Criteria
I'm seperating out my dataframe by the following criteria to make further analysis simpler. 
1. Seperate dataframe by genre
2. Seperate dataframe by decade 

In [ ]:
rt_df["YearB"].min()

In [ ]:
# I had an error here and had to strip the whitespace from my Year column in order to type the data as 'int'
rt_df["YearB"].isnull().values.any()

In [ ]:
rt_df["YearB"] = pd.to_numeric(rt_df["YearB"].astype(str).str.strip(), errors='coerce')

In [ ]:
print (rt_df["YearB"].tolist()[:20])

In [ ]:
rt_df.info()

In [ ]:
conditions = [
    (rt_df["YearB"] <= 1929), 
    (rt_df["YearB"] >= 1930) & (rt_df["YearB"] <= 1939), 
    (rt_df["YearB"] >= 1940) & (rt_df["YearB"] <= 1949),
    (rt_df["YearB"] >= 1950) & (rt_df["YearB"] <= 1959),
    (rt_df["YearB"] >= 1960) & (rt_df["YearB"] <= 1969),
    (rt_df["YearB"] >= 1970) & (rt_df["YearB"] <= 1979),
    (rt_df["YearB"] >= 1980) & (rt_df["YearB"] <= 1989),
    (rt_df["YearB"] >= 1990) & (rt_df["YearB"] <= 1999),
    (rt_df["YearB"] >= 2000) & (rt_df["YearB"] <= 2009),
    (rt_df["YearB"] >= 2010) & (rt_df["YearB"] <= 2019),
    (rt_df["YearB"] >= 2020) & (rt_df["YearB"] <= 2022)
]

values = ["1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

rt_df["Decade"] = np.select(conditions, values)

rt_df.head()

In [ ]:
rt_df["Genre"].unique()

In [ ]:
action_df = rt_df.loc[rt_df["Genre"] == "Action & Adventure"]

In [ ]:
action_df.head()

In [ ]:
animation_df = rt_df.loc[rt_df["Genre"] == "Animation"]
art_house_df = rt_df.loc[rt_df["Genre"] == "Art House & International"]
classics_df = rt_df.loc[rt_df["Genre"] == "Classics"]
comedy_df = rt_df.loc[rt_df["Genre"] == "Documentary"]
drama_df = rt_df.loc[rt_df["Genre"] == "Drama"]
doc_df = rt_df.loc[rt_df["Genre"] == "Documentary"]
horror_df = rt_df.loc[rt_df["Genre"] == "Horror"]
kids_df = rt_df.loc[rt_df["Genre"] == "Kids & Family"]
musical_df = rt_df.loc[rt_df["Genre"] == "Musical & Performing Arts"]
mystery_df = rt_df.loc[rt_df["Genre"] == "Mystery & Suspense"]
romance_df = rt_df.loc[rt_df["Genre"] == "Romance"]
scify_df = rt_df.loc[rt_df["Genre"] == "Science Fiction & Fantasy"]
special_df = rt_df.loc[rt_df["Genre"] == "Special Interest"]
sports_df = rt_df.loc[rt_df["Genre"] == "Sports & Fitness"]
tv_df = rt_df.loc[rt_df["Genre"] == "Television"]
western_df = rt_df.loc[rt_df["Genre"] == "Western"]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
fig8 = make_subplots(
    rows=5, cols=4,
    subplot_titles=("Action", "Animation", "Art House & Intern.", "Classics", "Comedy", "Drama", "Documentary" "Horror", "Kids", "Musicals", "Mystery", "Romance", "Scify", "Special", "Sports", "Tv", "Western"))

fig8.add_trace(go.Histogram(x=action_df["YearB"]),
              row=1, col=1)

fig8.add_trace(go.Histogram(x=animation_df["YearB"]),
              row=1, col=2)

fig8.add_trace(go.Histogram(x=art_house_df["YearB"]),
              row=1, col=3)

fig8.add_trace(go.Histogram(x=classics_df["YearB"]),
              row=1, col=4)

fig8.add_trace(go.Histogram(x=comedy_df["YearB"]),
              row=2, col=1)

fig8.add_trace(go.Histogram(x=drama_df["YearB"]),
              row=2, col=2)

fig8.add_trace(go.Histogram(x=doc_df["YearB"]), 
              row=2, col=3)

fig8.add_trace(go.Histogram(x=horror_df["YearB"]),
              row=2, col=4)

fig8.add_trace(go.Histogram(x=kids_df["YearB"]),
              row=3, col=1)

fig8.add_trace(go.Histogram(x=musical_df["YearB"]),
              row=3, col=2)

fig8.add_trace(go.Histogram(x=mystery_df["YearB"]),
              row=3, col=3)

fig8.add_trace(go.Histogram(x=romance_df["YearB"]),
              row=3, col=4)

fig8.add_trace(go.Histogram(x=scify_df["YearB"]),
              row=4, col=1)

fig8.add_trace(go.Histogram(x=special_df["YearB"]),
              row=4, col=2)

fig8.add_trace(go.Histogram(x=sports_df["YearB"]),
              row=4, col=3)

fig8.add_trace(go.Histogram(x=tv_df["YearB"]),
              row=4, col=4)

fig8.add_trace(go.Histogram(x=western_df["YearB"]),
              row=5, col=1)

fig8.show()

In [ ]:
early_twenties_df = rt_df.loc[rt_df["YearB"] <= 1929]

In [ ]:
fig5 = px.histogram(early_twenties_df, x="YearB")
fig5.show()

Now I know my seperation by decade has worked, so I will complete the rest of the seperations in the cell below. 

In [ ]:
thirties_df = rt_df.loc[(rt_df["YearB"] >= 1930) & (rt_df["YearB"] <= 1939)]
fourties_df = rt_df.loc[(rt_df["YearB"] >= 1940) & (rt_df["YearB"] <= 1949)]
fifties_df = rt_df.loc[(rt_df["YearB"] >= 1950) & (rt_df["YearB"] <= 1959)]
sixties_df = rt_df.loc[(rt_df["YearB"] >= 1960) & (rt_df["YearB"] <= 1969)]
seventies_df = rt_df.loc[(rt_df["YearB"] >= 1970) & (rt_df["YearB"] <= 1979)]
eighties_df = rt_df.loc[(rt_df["YearB"] >= 1980) & (rt_df["YearB"] <= 1989)]
ninties_df = rt_df.loc[(rt_df["YearB"] >= 1990) & (rt_df["YearB"] <= 1999)]
thousands_df = rt_df.loc[(rt_df["YearB"] >= 2000) & (rt_df["YearB"] <= 2009)]
twentyteens_df = rt_df.loc[(rt_df["YearB"] >= 2010) & (rt_df["YearB"] <= 2019)]
twentytwenties_df = rt_df.loc[(rt_df["YearB"] >= 2020) & (rt_df["YearB"] <= 2022)]

I will also add these decades as a column on my dataframe in order to utilize it later on. 

In [ ]:
thousands_df.head()

In [ ]:
fig7 = make_subplots(
    rows=4, cols=3,
    subplot_titles=("1920s", "1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s", "Every Year"))

fig7.add_trace(go.Histogram(x=early_twenties_df["YearB"]),
              row=1, col=1)

fig7.add_trace(go.Histogram(x=thirties_df["YearB"]),
              row=1, col=2)

fig7.add_trace(go.Histogram(x=fourties_df["YearB"]),
              row=1, col=3)

fig7.add_trace(go.Histogram(x=fifties_df["YearB"]),
              row=2, col=1)

fig7.add_trace(go.Histogram(x=sixties_df["YearB"]),
              row=2, col=2)

fig7.add_trace(go.Histogram(x=seventies_df["YearB"]),
              row=2, col=3)

fig7.add_trace(go.Histogram(x=eighties_df["YearB"]),
              row=3, col=1)

fig7.add_trace(go.Histogram(x=ninties_df["YearB"]),
              row=3, col=2)

fig7.add_trace(go.Histogram(x=thousands_df["YearB"]),
              row=3, col=3)

fig7.add_trace(go.Histogram(x=twentyteens_df["YearB"]),
              row=4, col=1)

fig7.add_trace(go.Histogram(x=twentytwenties_df["YearB"]),
              row=4, col=2)

fig7.add_trace(go.Histogram(x=rt_df["YearB"]),
              row=4, col=3)

fig7.show()

#  Answer Questions
Now that I have wrangled, organized, and explored my data I will being to answer my questions. 

1. Explore the year that had the most popular titles
2. Compare the data to see if any correlation exists
3. Find the most popular movies in each genre
4. The most popular movie overall

# 1. Which year had the most popular titles?
Answer: 2018

In [ ]:
rt_df["YearB"].value_counts()

In [ ]:
fig9 = px.histogram(rt_df, x="YearB")
fig9.show()

In [ ]:
fig10 = px.histogram(twentyteens_df, x="YearB")
fig10.show()

# 2.What correlations, if any, exist?
Answer: Number of Reviews and Year released have a slight positive relationship.
No onther clear correlations exist in this dataset.

In [ ]:
rt_df.corr()

Correlations varry from -1 to 1. 1 means there is a perfect relationship. 
(notice that this only happens when the matrix crosses itself)

The closer to zero we get (either negative or positive) the weaker the 
relationship between both variables. 

In this case, it seems that the Number of Reviews and Year of release have a somewhat moderate relationship. 

Correlation is not causation, but this could be worth exploring further. 

In [ ]:
fig11 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews")
fig11.show()

In [ ]:
fig12 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", color="Genre")
fig12.show()

Adding genre does not seem to create a more linear relationship, but I want to see this addition a little closer up to make sure that I haven't missed anything. 

In [ ]:
fig13 = make_subplots(
    rows=5, cols=4,
    subplot_titles=("Action", "Animation", "Art House & Intern.", "Classics", "Comedy", "Drama", "Docs" "Horror", "Kids", "Musicals", "Mystery", "Romance", "Scify", "Special", "Sports", "Tv", "Western"))

fig13.add_trace(go.Scatter(x=action_df["RatingTomatometer"], y=action_df["No. of Reviews"]),
              row=1, col=1)

fig13.add_trace(go.Scatter(x=animation_df["RatingTomatometer"], y=animation_df["No. of Reviews"]),
              row=1, col=2)

fig13.add_trace(go.Scatter(x=art_house_df["RatingTomatometer"], y=art_house_df["No. of Reviews"]),
              row=1, col=3)

fig13.add_trace(go.Scatter(x=classics_df["RatingTomatometer"], y=classics_df["No. of Reviews"]),
              row=1, col=4)

fig13.add_trace(go.Scatter(x=comedy_df["RatingTomatometer"], y=comedy_df["No. of Reviews"]),
              row=2, col=1)

fig13.add_trace(go.Scatter(x=drama_df["RatingTomatometer"], y=drama_df["No. of Reviews"]),
              row=2, col=2)

fig13.add_trace(go.Scatter(x=doc_df["RatingTomatometer"], y=doc_df["No. of Reviews"]),
               row=2, col=3)

fig13.add_trace(go.Scatter(x=horror_df["RatingTomatometer"], y=horror_df["No. of Reviews"]),
              row=2, col=4)

fig13.add_trace(go.Scatter(x=kids_df["RatingTomatometer"], y=kids_df["No. of Reviews"]),
              row=3, col=1)

fig13.add_trace(go.Scatter(x=musical_df["RatingTomatometer"], y=musical_df["No. of Reviews"]),
              row=3, col=2)

fig13.add_trace(go.Scatter(x=mystery_df["RatingTomatometer"], y=mystery_df["No. of Reviews"]),
              row=3, col=3)

fig13.add_trace(go.Scatter(x=romance_df["RatingTomatometer"], y=romance_df["No. of Reviews"]),
              row=3, col=4)

fig13.add_trace(go.Scatter(x=scify_df["RatingTomatometer"], y=scify_df["No. of Reviews"]),
              row=4, col=1)

fig13.add_trace(go.Scatter(x=special_df["RatingTomatometer"], y=special_df["No. of Reviews"]),
              row=4, col=2)

fig13.add_trace(go.Scatter(x=sports_df["RatingTomatometer"], y=sports_df["No. of Reviews"]),
              row=4, col=3)

fig13.add_trace(go.Scatter(x=tv_df["RatingTomatometer"], y=tv_df["No. of Reviews"]),
              row=4, col=4)

fig13.add_trace(go.Scatter(x=western_df["RatingTomatometer"], y=western_df["No. of Reviews"]),
              row=5, col=1)

fig13.show()

In [ ]:
fig14 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", facet_col="Genre", facet_col_wrap=2, width=800, height=2400)
fig14.update_layout(title="Ratings vs Number of Reviews")
fig14.show()

What I gather from these relationships (or lack thereof), is that Sports & Fitness, Television, and Western are less popular genres than the others and the movies with fewer reviewers were added to the top '100'. 

In [ ]:
!pip install statsmodels

In [ ]:
fig15 = px.scatter(rt_df, x="YearB", y="No. of Reviews", color="Decade", trendline="lowess", trendline_scope="overall", trendline_color_override="black")
fig15.show()

In [ ]:
fig16 = px.scatter(rt_df, x="YearB", y="No. of Reviews", facet_col= "Genre", facet_col_wrap= 3, trendline="ols", height=1200)
fig16.show()

Not all of these lines fit perfectly, but it does seem that the newer the release date, the more likely it is that the Number of Reviews will also be higher. This makes sense as the entertainment industry continues to grow. 

# Question 3: Find the most popular movies in each genre
Answer: 
Action & Adventure: Black Panther (2018)	
Animation: Toy Story 4	(2019)	
Art House & International: Parasite	(2019)	
Classics: It Happened One Night	(1934)	
Documentary: Won't You Be My Neighbor?	(2018)
Drama: Citizen Kane	(1941)	
Horror: Us	(2019)	
Kids & Family: The Wizard of Oz	(1939)	
Musical & Performing Arts: The Wizard of Oz	(1939)
Mystery & Suspense: Citizen Kane (1941)	
Romance: It Happened One Night	(1934)
Science Fiction & Fantasy: The Wizard of Oz	(1939)	
Special Interest: The Bride of Frankenstein	(1935)	
Sports & Fitness: Murderball (2005)
Television: Life Itself	(2014)	
Western: The Treasure of the Sierra Madre (1948)

In [ ]:
fig6 = px.scatter(rt_df, x="RatingTomatometer", y="No. of Reviews", color="Genre")
fig6.show()

In [ ]:
rt_df.sort_values("RatingTomatometer", ascending=False).head()

In [ ]:
rt_df.sort_values("No. of Reviews", ascending=False).head()

Rotten Tomatoes has some movies listed under several genres, I'll be sure to address that below.

In [ ]:
# I am going to remind myself what genres I will be evaluating 
rt_df["Genre"].unique()

In [ ]:
top_action = action_df.sort_values("Rank").head(10)
top_animation = animation_df.sort_values("Rank").head(10)
top_art = art_house_df.sort_values("Rank").head(10)
top_classics = classics_df.sort_values("Rank").head(10)
top_comedy = comedy_df.sort_values("Rank").head(10)
top_doc = doc_df.sort_values("Rank").head(10)
top_drama = drama_df.sort_values("Rank").head(10)
top_horror = horror_df.sort_values("Rank").head(10)
top_kids = kids_df.sort_values("Rank").head(10)
top_musical = musical_df.sort_values("Rank").head(10)
top_mystery = mystery_df.sort_values("Rank").head(10)
top_romance = romance_df.sort_values("Rank").head(10)
top_science = scify_df.sort_values("Rank").head(10)
top_special = special_df.sort_values("Rank").head(10)
top_sports = sports_df.sort_values("Rank").head(10)
top_tv = tv_df.sort_values("Rank").head(10)
top_west = western_df.sort_values("Rank").head(10)

In [ ]:
display(top_action)
display(top_animation)
display(top_art)
display(top_classics)
display(top_comedy)
display(top_doc)
display(top_drama)
display(top_horror)
display(top_kids)
display(top_musical)
display(top_mystery)
display(top_romance)
display(top_science)
display(top_special)
display(top_sports)
display(top_tv)
display(top_west)

In [ ]:
frames = [top_action, top_animation, top_art, top_classics, top_comedy, top_doc, top_drama, top_horror, top_kids, top_musical, top_mystery, top_romance, top_science, top_special, top_sports, top_tv, top_west]

In [ ]:
top_result = pd.concat(frames)
top_result

In [ ]:
fig17 = px.bar(top_result, x="Rank", y="Scaled Reviews", color="Decade")
fig17.show()

# 4. The favorite movie overall 
(According to "Rotten Tomatoes" critics) 
The highest ranking movie with the best 'Tomatometer' score is "Black Panther (2018)": Action and Adventure. 

In [ ]:
first = top_result.loc[top_result["Rank"] == 1]
first

In [ ]:
first.sort_values(by=["Scaled Reviews"], ascending=False)

Sorting by the 'Scaled number of reviews' only tells me how often this movie had a review written about it. I need to focus on the Tomatometer ranking in order to find the most popular title. 

In [ ]:
rt_df[['Scaled Tomatometer']] = scaler.fit_transform(rt_df[["RatingTomatometer"]])

rt_df.head()

In [ ]:
rt_df.sort_values(by=["Scaled Tomatometer"], ascending=False)

This has not been as accurate an image as I would have liked so I will take this dataframe and all of the values with a 'Scaled Tomatometer' at 1 and then sort these values by 'Scaled Reviews'. 

In [ ]:
one_tomato = rt_df.loc[rt_df["Scaled Tomatometer"] == 1]
one_tomato

In [ ]:
one_tomato.sort_values(by=["Scaled Reviews"], ascending=False)

This still isn't quite the results I was hoping for, So I am going to further limit my dataframe. 

In [ ]:
one_tomato = rt_df.loc[(rt_df["Scaled Tomatometer"] >= .9) & (rt_df["Scaled Reviews"] >= .9)]
one_tomato

In [ ]:
one_tomato.sort_values(by=["Scaled Tomatometer"], ascending=False)

In [ ]:
one_tomato.sort_values(by=["Scaled Reviews"], ascending=False)

# Future Projects 
Connected to this project, I would like to explore some of this data further. It will require me to scrape some additional websites and track down a little more information than the current parameters of this project needs. Here is the list of information I would like to gather. 
1. Audience ratings 
2. The money spent on production compared to money earned 
3. The money spent on advertising compared to money earned 
4. The money spent on production compared to audience ratings 